# Shelve

## 1. Introduction
The built-in shelve module provides a simple, persistent, dictionary-like object: you can store and retrieve Python objects by key and have them backed to disk automatically. Underneath it uses `dbm` and `pickle`, so any pickleable object can be saved with minimal boilerplate.

Use cases in SafeChess:

* Cache engine results by (`fen`, `depth`, `multipv`) keys
* Persist small lookup tables or thresholds between runs
* Quick prototyping of on-disk storage without a full database

## 2. Paradigm & Key Objects

|Concept|	Description|
|---|---|
|Shelf|	The main object: works like a `dict` but persists to a file.|
|Keys|	Must be strings (as DBM keys are byte strings under the hood).|
|Values|	Any pickleable Python object (lists, dicts, custom classes).|
|`writeback`|	If `True`, caches all accessed entries in memory and writes back on close—useful for mutating nested structures, but memory-hungry.|
|`sync()`|	Flush the in-memory cache to disk without closing.|
|Underlying dbm|	Backed by any supported DBM implementation (`dbm.gnu`, `dbm.ndbm`, etc.).|

```python
import shelve
```
`open` → returns a shelf object

`shelf[key]` = value → stores (pickles) value

`value = shelf[key]` → loads (unpickles) value

`del shelf[key]` → removes entry

`shelf.close()` → flushes and closes the file

## 3. Key Functionality

### Opening a Shelf

In [1]:
import shelve

# Open (and create if missing) a shelf file named "cache.db"
# flag="c": create if needed; flag="r": read-only; flag="n": new; flag="w": read/write
shelf = shelve.open("cache.db", flag="c", writeback=False)

### Basic Operations

In [2]:
# Store a value
shelf["foo"] = {"bar": 123, "baz": [1,2,3]}

# Retrieve a value
data = shelf["foo"]
print(data["bar"])  # 123

# Check for a key
if "foo" in shelf:
    print("Found it!")

# Delete a key
del shelf["foo"]


123
Found it!


### Iteration and Listing

In [3]:
# Keys, values, items just like a dict
for key in shelf:
    print(key)

for key, value in shelf.items():
    print(key, "→", value)


### `writeback` and `sync`

In [4]:
# If you need to mutate a nested object in place:
shelf = shelve.open("cache.db", writeback=True)
shelf["nested"] = {"counter": 0}

# Mutate the stored dict directly
shelf["nested"]["counter"] += 1

# Force write to disk now
shelf.sync()

# At the end
shelf.close()


Warning: `writeback=True` keeps all accessed items in RAM until close or sync; avoid for large datasets.

### Closing the Shelf
Always close to make sure that data is flushed. We can also use a context manager pattern.

In [5]:
shelf = shelve.open("cache.db", writeback=True)
shelf.close()

In [6]:
with shelve.open("cache.db", writeback=True) as shelf:
    # Do something with the shelf
    pass  # No need to call close() explicitly

## 4. Gotchas
* String-only keys: non-string keys will be coerced or error.
* Pickle security: don’t open untrusted shelves (they can execute arbitrary code when unpickled).
* Concurrent writes: DBM backends often don’t support multiple writers; avoid simultaneous writes or use a lock.
* Performance: good for small-to-medium datasets; large volumes might warrant SQLite or a proper key-value store.